# **BASIC OF FLASK**

In [1]:
# 1. Create a Flask app that displays "Hello, World!" on the homepage.
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
# 2. Build a Flask app with static HTML pages and navigate between them.
# index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Home</title>
</head>
<body>
    <h1>Welcome to the Home Page</h1>
    <p>This is the home page.</p>
    <a href="/about">Go to About Page</a>
</body>
</html>



# About.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>About</title>
</head>
<body>
    <h1>About Us</h1>
    <p>This is the about page.</p>
    <a href="/">Go to Home Page</a>
</body>
</html>



# app.py
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/about')
def about():
    return render_template('about.html')

if __name__ == '__main__':
    app.run(debug=True)


# all file name is give above code save it as it is and done

In [ ]:
# 3. Develop a Flask app that uses URL parameters to display dynamic content.
from flask import Flask

app = Flask(__name__)

@app.route('/hello/<name>')
def hello_name(name):
    return f'Hello, {name}!'

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
# 4. Create a Flask app with a form that accepts user input and displays it.

# app.py
from flask import Flask, render_template, request

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/submit', methods=['POST'])
def submit():
    if request.method == 'POST':
        user_input = request.form['user_input']
        return render_template('result.html', user_input=user_input)

if __name__ == '__main__':
    app.run(debug=True)



# index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Flask Form Example</title>
</head>
<body>
    <h1>Enter your input:</h1>
    <form action="/submit" method="post">
        <input type="text" name="user_input">
        <input type="submit" value="Submit">
    </form>
</body>
</html>


# result.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Flask Form Result</title>
</head>
<body>
    <h1>You entered:</h1>
    <p>{{ user_input }}</p>
</body>
</html>


In [ ]:
# 5. Implement user sessions in a Flask app to store and display user-specific data.


# App.py
from flask import Flask, render_template, request, session, redirect, url_for

app = Flask(__name__)
app.secret_key = 'your_secret_key'

@app.route('/')
def index():
    if 'username' in session:
        return 'Logged in as ' + session['username'] + '<br>' + \
            "<b><a href = '/logout'>click here to log out</a></b>"
    return "You are not logged in <br><a href = '/login'></b>" + \
        "click here to log in</b></a>"

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        session['username'] = request.form['username']
        return redirect(url_for('index'))
    return '''
        <form action = "" method = "post">
            <p><input type = text name = username/></p>
            <p<<input type = submit value = Login/></p>
        </form>
    '''

@app.route('/logout')
def logout():
    session.pop('username', None)
    return redirect(url_for('index'))

if __name__ == '__main__':
    app.run(debug=True)


# **Intermediate Flask Topics:**

In [ ]:
# 6. Build a Flask app that allows users to upload files and display them on the website.
# App.py


import os
from flask import Flask, render_template, request, redirect, url_for

UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'}

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = file.filename
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            return redirect(url_for('uploaded_file', filename=filename))
    return render_template('index.html')

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return render_template('uploaded.html', filename=filename)

if __name__ == '__main__':
    app.run(debug=True)


# Index.html

<!DOCTYPE html>
<html>
<head>
    <title>File Upload</title>
</head>
<body>
    <h1>Upload File</h1>
    <form method="POST" enctype="multipart/form-data" action="/upload">
        <input type="file" name="file">
        <input type="submit" value="Upload">
    </form>
</body>
</html>




# uploaded.html
<!DOCTYPE html>
<html>
<head>
    <title>Uploaded File</title>
</head>
<body>
    <h1>Uploaded File</h1>
    <img src="{{ url_for('static', filename='uploads/' + filename) }}" alt="Uploaded file">
</body>
</html>


In [ ]:
# 7. Integrate a SQLite database with Flask to perform CRUD operations on a list of items.

# App.py
from flask import Flask, render_template, request, redirect, url_for
import sqlite3

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'
app.config['DATABASE'] = 'items.db'

def get_db_connection():
    conn = sqlite3.connect(app.config['DATABASE'])
    conn.row_factory = sqlite3.Row
    return conn

def init_db():
    conn = get_db_connection()
    with app.open_resource('schema.sql', mode='r') as f:
        conn.cursor().executescript(f.read())
    conn.commit()
    conn.close()

@app.route('/')
def index():
    conn = get_db_connection()
    items = conn.execute('SELECT * FROM items').fetchall()
    conn.close()
    return render_template('index.html', items=items)

@app.route('/add', methods=['POST'])
def add_item():
    if request.method == 'POST':
        name = request.form['name']
        if not name:
            return redirect(url_for('index'))
        conn = get_db_connection()
        conn.execute('INSERT INTO items (name) VALUES (?)', (name,))
        conn.commit()
        conn.close()
        return redirect(url_for('index'))

@app.route('/delete/<int:item_id>', methods=['POST'])
def delete_item(item_id):
    conn = get_db_connection()
    conn.execute('DELETE FROM items WHERE id = ?', (item_id,))
    conn.commit()
    conn.close()
    return redirect(url_for('index'))

if __name__ == '__main__':
    init_db()
    app.run(debug=True)



# schema.sql
CREATE TABLE IF NOT EXISTS items (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL
);



# templates.html
<!DOCTYPE html>
<html>
<head>
    <title>Item List</title>
</head>
<body>
    <h1>Item List</h1>
    <ul>
        {% for item in items %}
        <li>{{ item['name'] }} <form method="post" action="/delete/{{ item['id'] }}"><button type="submit">Delete</button></form></li>
        {% endfor %}
    </ul>
    <h2>Add New Item</h2>
    <form method="post" action="/add">
        <input type="text" name="name">
        <button type="submit">Add</button>
    </form>
</body>
</html>



In [ ]:
# 8. Implement user authentication and registration in a Flask app using Flask-Login.
# pip install Flask-Login

# app.py
from flask import Flask, render_template, request, redirect, url_for
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user

app = Flask(__name__)
app.secret_key = 'your_secret_key'

# Set up Flask-Login
login_manager = LoginManager()
login_manager.init_app(app)

# Dummy User Model (Replace this with your SQLAlchemy User model)
class User(UserMixin):
    def __init__(self, id):
        self.id = id

# Dummy user database (Replace this with your actual user database)
users = {'user_id': {'password': 'password'}}

@login_manager.user_loader
def load_user(user_id):
    return User(user_id)

@app.route('/')
def index():
    return 'Hello, please login <a href="/login">here</a>.'

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        user_id = request.form['user_id']
        password = request.form['password']
        if user_id in users and password == users[user_id]['password']:
            user = User(user_id)
            login_user(user)
            return redirect(url_for('dashboard'))
    return render_template('login.html')

@app.route('/dashboard')
@login_required
def dashboard():
    return f'Welcome {current_user.id}, you are logged in! <br> <a href="/logout">Logout</a>'

@app.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for('index'))

if __name__ == '__main__':
    app.run(debug=True)





# login.html

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Login</title>
</head>
<body>
    <h1>Login</h1>
    <form method="post" action="/login">
        <label for="user_id">User ID:</label><br>
        <input type="text" id="user_id" name="user_id"><br>
        <label for="password">Password:</label><br>
        <input type="password" id="password" name="password"><br><br>
        <input type="submit" value="Login">
    </form>
</body>
</html>





In [ ]:
# 9. Create a RESTful API using Flask to perform CRUD operations on resources like books or movies.

# app.py

from flask import Flask, jsonify, request

app = Flask(__name__)

books = [
    {"id": 1, "title": "Book 1", "author": "Author 1"},
    {"id": 2, "title": "Book 2", "author": "Author 2"}
]

# Get all books
@app.route('/books', methods=['GET'])
def get_books():
    return jsonify(books)

# Get a single book
@app.route('/books/<int:book_id>', methods=['GET'])
def get_book(book_id):
    book = next((book for book in books if book['id'] == book_id), None)
    if book:
        return jsonify(book)
    return jsonify({"message": "Book not found"}), 404

# Create a new book
@app.route('/books', methods=['POST'])
def create_book():
    data = request.json
    if 'title' in data and 'author' in data:
        new_book = {
            "id": len(books) + 1,
            "title": data['title'],
            "author": data['author']
        }
        books.append(new_book)
        return jsonify(new_book), 201
    return jsonify({"message": "Title and author are required"}), 400

# Update a book
@app.route('/books/<int:book_id>', methods=['PUT'])
def update_book(book_id):
    data = request.json
    book = next((book for book in books if book['id'] == book_id), None)
    if book:
        book.update(data)
        return jsonify(book), 200
    return jsonify({"message": "Book not found"}), 404

# Delete a book
@app.route('/books/<int:book_id>', methods=['DELETE'])
def delete_book(book_id):
    global books
    initial_length = len(books)
    books = [book for book in books if book['id'] != book_id]
    if len(books) < initial_length:
        return jsonify({"message": "Book deleted"}), 200
    return jsonify({"message": "Book not found"}), 404

if __name__ == '__main__':
    app.run(debug=True)



# bash---app

# Get all books
curl -X GET http://127.0.0.1:5000/books

# Get a single book (replace <book_id> with an actual book ID)
curl -X GET http://127.0.0.1:5000/books/<book_id>

# Create a new book
curl -X POST -H "Content-Type: application/json" -d '{"title":"New Book", "author":"New Author"}' http://127.0.0.1:5000/books

# Update an existing book (replace <book_id> with an actual book ID)
curl -X PUT -H "Content-Type: application/json" -d '{"title":"Updated Book"}' http://127.0.0.1:5000/books/<book_id>

# Delete a book by ID (replace <book_id> with an actual book ID)
curl -X DELETE http://127.0.0.1:5000/books/<book_id>


In [ ]:
# 10. Design a Flask app with proper error handling for 404 and 500 errors.


# app.py


from flask import Flask, render_template

app = Flask(__name__)

# Custom error handlers
@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404

@app.errorhandler(500)
def internal_server_error(e):
    return render_template('500.html'), 500

# Routes
@app.route('/')
def index():
    return 'Welcome to the Flask App'

@app.route('/error404')
def trigger_404():
    # Triggering a 404 error manually for testing
    abort(404)

@app.route('/error500')
def trigger_500():
    # Triggering a 500 error manually for testing
    1 / 0

if __name__ == '__main__':
    app.run(debug=True)



# 400.html

<!DOCTYPE html>
<html>
<head>
    <title>404 Not Found</title>
</head>
<body>
    <h1>404 Not Found</h1>
    <p>The requested page could not be found.</p>
</body>
</html>


# 500.html
<!DOCTYPE html>
<html>
<head>
    <title>500 Internal Server Error</title>
</head>
<body>
    <h1>500 Internal Server Error</h1>
    <p>An unexpected error occurred on the server.</p>
</body>
</html>


# **Real-time Development:**

In [ ]:
# 11. Create a real-time chat application using Flask-SocketIO.
# App.py

from flask import Flask, render_template
from flask_socketio import SocketIO, emit

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'
socketio = SocketIO(app)

@app.route('/')
def index():
    return render_template('index.html')

@socketio.on('message')
def handle_message(message):
    emit('message', message, broadcast=True)

if __name__ == '__main__':
    socketio.run(app)



# index.html

<!DOCTYPE html>
<html>
<head>
    <title>Real-time Chat</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.4.1/socket.io.js" integrity="sha512-QH6F2jDgsr6xFX46OO9YOqQ3RVGCg3gpJO2sP49Fd5UCOnK7O/eGfvfRkJ8VhXO0h9f+HsU/y/+iSUGQlb/2vQ==" crossorigin="anonymous" referrerpolicy="no-referrer"></script>
    <script type="text/javascript">
        var socket = io.connect('http://' + document.domain + ':' + location.port);

        socket.on('connect', function() {
            socket.send('User connected');
        });

        socket.on('message', function(msg) {
            var li = document.createElement('li');
            li.innerHTML = msg;
            document.getElementById("messages").appendChild(li);
        });

        function sendMessage() {
            var messageInput = document.getElementById("messageInput");
            var message = messageInput.value;
            socket.send(message);
            messageInput.value = '';
        }
    </script>
</head>
<body>
    <h1>Real-time Chat</h1>
    <ul id="messages"></ul>
    <input type="text" id="messageInput">
    <button onclick="sendMessage()">Send</button>
</body>
</html>


In [ ]:
# 12. Build a Flask app that updates data in real-time using WebSocket connections.

# app.py
from flask import Flask, render_template
from flask_socketio import SocketIO, emit
import random
import threading
import time

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'
socketio = SocketIO(app)

data = {'value': 0}

def update_data():
    global data
    while True:
        data['value'] = random.randint(0, 100)
        socketio.emit('data_update', data['value'])
        time.sleep(1)

@app.route('/')
def index():
    return render_template('index.html', initial_value=data['value'])

@socketio.on('connect')
def handle_connect():
    emit('data_update', data['value'])

if __name__ == '__main__':
    threading.Thread(target=update_data).start()
    socketio.run(app)



# index.html
<!DOCTYPE html>
<html>
<head>
    <title>Real-time Data Update</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.4.1/socket.io.js" integrity="sha512-QH6F2jDgsr6xFX46OO9YOqQ3RVGCg3gpJO2sP49Fd5UCOnK7O/eGfvfRkJ8VhXO0h9f+HsU/y/+iSUGQlb/2vQ==" crossorigin="anonymous" referrerpolicy="no-referrer"></script>
    <script type="text/javascript">
        var socket = io.connect('http://' + document.domain + ':' + location.port);

        socket.on('data_update', function(value) {
            document.getElementById("dataValue").innerText = value;
        });
    </script>
</head>
<body>
    <h1>Real-time Data Update</h1>
    <p>Data Value: <span id="dataValue">{{ initial_value }}</span></p>
</body>
</html>


In [ ]:
# 13. Implement notifications in a Flask app using websockets to notify users of updates.

# app.py
from flask import Flask, render_template
from flask_socketio import SocketIO, emit
import threading
import time

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'
socketio = SocketIO(app)

notifications = []

def send_notifications():
    while True:
        if notifications:
            notification = notifications.pop(0)
            socketio.emit('notification', notification)
        time.sleep(1)

@app.route('/')
def index():
    return render_template('index.html')

@socketio.on('connect')
def handle_connect():
    emit('notification', 'Connected to notifications')

@app.route('/notify')
def notify():
    notifications.append('New update available!')
    return 'Notification sent'

if __name__ == '__main__':
    threading.Thread(target=send_notifications).start()
    socketio.run(app)



# index.html
<!DOCTYPE html>
<html>
<head>
    <title>Notifications</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.4.1/socket.io.js" integrity="sha512-QH6F2jDgsr6xFX46OO9YOqQ3RVGCg3gpJO2sP49Fd5UCOnK7O/eGfvfRkJ8VhXO0h9f+HsU/y/+iSUGQlb/2vQ==" crossorigin="anonymous" referrerpolicy="no-referrer"></script>
    <script type="text/javascript">
        var socket = io.connect('http://' + document.domain + ':' + location.port);

        socket.on('notification', function(notification) {
            var li = document.createElement('li');
            li.innerHTML = notification;
            document.getElementById("notifications").appendChild(li);
        });
    </script>
</head>
<body>
    <h1>Notifications</h1>
    <ul id="notifications"></ul>
</body>
</html>
